In [ ]:
# train_model_colab.py
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2
from google.colab import drive


## Mount Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Image and Steering Wheel Data from the Provided Directory

In [ ]:
def load_data(df, img_folder):
    X = []
    y = []
    for _, row in df.iterrows():
        img_filename = row['file_name']
        if not img_filename.lower().endswith('.jpg'):
            img_filename += '.jpg'
        img_path = os.path.join(img_folder, img_filename)

        # Load image in color
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Failed to load image: {img_path}")
            continue

        # Convert to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Resize and normalize
        img = cv2.resize(img, (96, 96)).astype('float32') / 255.0

        X.append(img)
        y.append(row['class_name'])

    X = np.expand_dims(np.array(X), axis=-1)  # shape: (N, 96, 96, 1)
    y = np.array(y, dtype='float32')
    return X, y

## Main Data Set

In [ ]:
# Main dataset
CSV_PATH = "/content/drive/MyDrive/self_driving car/td_bt2_0502_track1/controller_data.csv"
IMG_FOLDER = "/content/drive/MyDrive/self_driving car/td_bt2_0502_track1"
df = pd.read_csv(CSV_PATH)
df['file_name'] = df['file_name'].astype(str)
X, y = load_data(df, IMG_FOLDER)

print("Shapes before combining:")
print("X:", X.shape)
print("y:", y.shape)


Shapes before combining:
X: (2091, 96, 96, 1)
y: (2091,)


## Additional Path Correction Dataset

In [ ]:
# Additional dataset
CSV_PATH_ADD = "/content/drive/MyDrive/self_driving car/td_bt2_0502_0630_correction/controller_data.csv"
IMG_FOLDER_ADD = "/content/drive/MyDrive/self_driving car/td_bt2_0502_0630_correction"
df_add = pd.read_csv(CSV_PATH_ADD)
df_add['file_name'] = df_add['file_name'].astype(str)
X_add, y_add = load_data(df_add, IMG_FOLDER_ADD)

# Combine datasets
X = np.concatenate((X, X_add))
y = np.concatenate((y, y_add))

print("Shapes after combining:")
print("X:", X.shape)
print("y:", y.shape)


Shapes after combining:
X: (3379, 96, 96, 1)
y: (3379,)


##  Mooooore Data

In [ ]:
# Additional dataset
CSV_PATH_ADD = "/content/drive/MyDrive/self_driving car/td_bt2_0502_0630_track/controller_data.csv"
IMG_FOLDER_ADD = "/content/drive/MyDrive/self_driving car/td_bt2_0502_0630_track"
df_add = pd.read_csv(CSV_PATH_ADD)
df_add['file_name'] = df_add['file_name'].astype(str)
X_add, y_add = load_data(df_add, IMG_FOLDER_ADD)

# Combine datasets
X = np.concatenate((X, X_add))
y = np.concatenate((y, y_add))

print("Shapes after combining 2:")
print("X:", X.shape)
print("y:", y.shape)


Shapes after combining 2:
X: (7400, 96, 96, 1)
y: (7400,)


## Even More Data on Tricky Corners

In [ ]:
CSV_PATH_ADD = "/content/drive/MyDrive/self_driving car/saved_images/controller_data.csv"
IMG_FOLDER_ADD = "/content/drive/MyDrive/self_driving car/saved_images"
df_add = pd.read_csv(CSV_PATH_ADD)
df_add['file_name'] = df_add['file_name'].astype(str)
X_add, y_add = load_data(df_add, IMG_FOLDER_ADD)

# Combine datasets
X = np.concatenate((X, X_add))
y = np.concatenate((y, y_add))

print("Shapes after combining 3:")
print("X:", X.shape)
print("y:", y.shape)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print("Shapes after combining and splitting:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)

Shapes after combining 3:
X: (8494, 96, 96, 1)
y: (8494,)
Shapes after combining and splitting:
X_train: (7644, 96, 96, 1)
y_train: (7644,)
X_val: (850, 96, 96, 1)
y_val: (850,)


## Define CNN Model

In [ ]:
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(96, 96, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu', ),
    layers.Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 94, 94, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       495,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 500,481 (1.91 MB)

 Trainable params: 500,481 (1.91 MB)

 Non-trainable params: 0 (0.00 B)

## Train Model

In [ ]:
# Train the model and save best weights
checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True)
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_cb]
)

# Save training and validation metrics to CSV
metrics_df = pd.DataFrame({
    'epoch': range(1, len(history.history['loss']) + 1),
    'train_loss_mse': history.history['loss'],
    'train_mae': history.history['mae'],
    'val_loss_mse': history.history['val_loss'],
    'val_mae': history.history['val_mae']
})
metrics_df.to_csv("training_metrics11.csv", index=False)

# Load best weights before conversion
model.load_weights("best_model.h5")

# Convert to float32 TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model_fp32.tflite", "wb") as f:
    f.write(tflite_model)

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1482 - mae: 0.2832

239/239 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.1480 - mae: 0.2829 - val_loss: 0.0518 - val_mae: 0.1430
Epoch 2/100
221/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0523 - mae: 0.1487

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0520 - mae: 0.1483 - val_loss: 0.0384 - val_mae: 0.1208
Epoch 3/100
230/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0413 - mae: 0.1296

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0413 - mae: 0.1295 - val_loss: 0.0349 - val_mae: 0.1133
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0350 - mae: 0.1193 - val_loss: 0.0367 - val_mae: 0.1194
Epoch 5/100
222/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0320 - mae: 0.1140

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0320 - mae: 0.1139 - val_loss: 0.0317 - val_mae: 0.1115
Epoch 6/100
227/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0294 - mae: 0.1096

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0295 - mae: 0.1096 - val_loss: 0.0314 - val_mae: 0.1095
Epoch 7/100
232/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0286 - mae: 0.1073

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0287 - mae: 0.1074 - val_loss: 0.0293 - val_mae: 0.1051
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0260 - mae: 0.1039 - val_loss: 0.0296 - val_mae: 0.1043
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0260 - mae: 0.1024 - val_loss: 0.0303 - val_mae: 0.1020
Epoch 10/100
233/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0256 - mae: 0.1010

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0256 - mae: 0.1011 - val_loss: 0.0290 - val_mae: 0.1034
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0236 - mae: 0.0975 - val_loss: 0.0303 - val_mae: 0.1054
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0231 - mae: 0.0957 - val_loss: 0.0306 - val_mae: 0.1081
Epoch 13/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0221 - mae: 0.0947 - val_loss: 0.0291 - val_mae: 0.1046
Epoch 14/100
226/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0207 - mae: 0.0953

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0208 - mae: 0.0954 - val_loss: 0.0287 - val_mae: 0.1049
Epoch 15/100
230/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0214 - mae: 0.0922

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0214 - mae: 0.0922 - val_loss: 0.0284 - val_mae: 0.1025
Epoch 16/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0195 - mae: 0.0904 - val_loss: 0.0308 - val_mae: 0.1102
Epoch 17/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0194 - mae: 0.0903 - val_loss: 0.0298 - val_mae: 0.1041
Epoch 18/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0197 - mae: 0.0897 - val_loss: 0.0292 - val_mae: 0.1039
Epoch 19/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0188 - mae: 0.0870 - val_loss: 0.0294 - val_mae: 0.1047
Epoch 20/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0185 - mae: 0.0875 - val_loss: 0.0293 - val_mae: 0.1031
Epoch 21/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0188 - mae: 0.0859 - val_loss: 0.0297 - val_mae: 0.1048
Epoch 22/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0181 - mae: 0.0859 - val_loss: 0.0294 - val_mae: 0.1051
Epoch 23/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

## Quick Sanity Check

In [ ]:
# Predict on the validation set
y_pred = model.predict(X_val)

# Create a DataFrame to display the results
results_df = pd.DataFrame({'Actual': y_val, 'Predicted': y_pred.flatten()})
results_df['Difference'] = results_df['Actual'] - results_df['Predicted']

# Display the results
results_df


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


,Actual,Predicted,Difference
0,-0.047333,-0.079715,0.032382
1,-0.063293,-0.079715,0.016422
2,0.956085,0.780057,0.176028
3,-0.066315,-0.079715,0.013400
4,-0.076569,-0.079715,0.003146
...,...,...,...
845,0.201630,0.127929,0.073701
846,-0.063110,-0.079715,0.016605
847,-0.014832,-0.079715,0.064883
848,-0.216217,-0.079715,-0.136502


## Quantize to INT8

In [ ]:
# Load best weights
model.load_weights("best_model.h5")

# Randomly sample 100 representative inputs across the dataset
sample_idx = np.random.choice(len(X_train), size=2500, replace=False)

def representative_data_gen():
    for i in sample_idx:
        img = X_train[i]  # shape: (96, 96, 1), float32 in [0.0, 1.0]
        img = img * 255.0  # Rescale to [0, 255]
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        yield [img.astype(np.float32)]  # Must stay float32 for calibration

# Setup TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert model
tflite_quant_model = converter.convert()

# Save model
with open("model_int8_perchannel.tflite", "wb") as f:
    f.write(tflite_quant_model)

Saved artifact at '/tmp/tmpb_93pgu1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132628191074960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191075728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191071504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191075152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191071888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191078992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191080336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132628191080144: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_path="model_int8_perchannel.tflite")
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Extract input/output quantization parameters
scale_in, zero_point_in = input_details[0]['quantization']
scale_out, zero_point_out = output_details[0]['quantization']

# Predict on the validation set
predictions = []
for i in range(len(X_val)):
    # Prepare input: convert from normalized [0.0, 1.0] to uint8 quantized
    img = (X_val[i:i+1] * 255.0).astype(np.float32)
    img_quant = np.clip(np.round(img / scale_in + zero_point_in), 0, 255).astype(np.uint8)

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], img_quant)
    interpreter.invoke()
    output_quant = interpreter.get_tensor(output_details[0]['index'])

    # Dequantize output to float32
    pred = (output_quant.astype(np.float32) - zero_point_out) * scale_out/255
    predictions.append(pred[0][0])

# Compile results into a DataFrame
results_df = pd.DataFrame({
    'Actual': y_val,
    'Predicted': predictions
})
results_df['Difference'] = results_df['Actual'] - results_df['Predicted']

# Show results
results_df

,Actual,Predicted,Difference
0,-0.047333,0.212342,-0.259675
1,-0.063293,0.000000,-0.063293
2,0.956085,0.445919,0.510166
3,-0.066315,0.000000,-0.066315
4,-0.076569,0.000000,-0.076569
...,...,...,...
845,0.201630,0.193762,0.007868
846,-0.063110,0.000000,-0.063110
847,-0.014832,0.106171,-0.121003
848,-0.216217,0.138022,-0.354239


In [ ]:
# Load the quantized TFLite model (INT8)
interpreter_int8 = tf.lite.Interpreter(model_path="model_int8_perchannel.tflite")
interpreter_int8.allocate_tensors()

input_details_int8 = interpreter_int8.get_input_details()
output_details_int8 = interpreter_int8.get_output_details()

scale_in_int8, zero_point_in_int8 = input_details_int8[0]['quantization']
scale_out_int8, zero_point_out_int8 = output_details_int8[0]['quantization']

# Predict on the validation set (INT8)
predictions_int8 = []
for i in range(len(X_val)):
    img = (X_val[i:i+1] * 255.0).astype(np.float32)
    img_quant = np.clip(np.round(img / scale_in_int8 + zero_point_in_int8), 0, 255).astype(np.uint8)
    interpreter_int8.set_tensor(input_details_int8[0]['index'], img_quant)
    interpreter_int8.invoke()
    output_quant = interpreter_int8.get_tensor(output_details_int8[0]['index'])
    pred = (output_quant.astype(np.float32) - zero_point_out_int8) * scale_out_int8/255
    predictions_int8.append(pred[0][0])

predictions_int8 = np.array(predictions_int8)
mae_int8 = np.mean(np.abs(y_val - predictions_int8))
mse_int8 = mean_squared_error(y_val, predictions_int8)

print(f"MAE (INT8): {mae_int8:.6f}")
print(f"MSE (INT8): {mse_int8:.6f}")

# Predict on the validation set (FP32)
y_pred_fp32 = model.predict(X_val).flatten()
mae_fp32 = np.mean(np.abs(y_val - y_pred_fp32))
mse_fp32 = mean_squared_error(y_val, y_pred_fp32)

print(f"MAE (FP32): {mae_fp32:.6f}")
print(f"MSE (FP32): {mse_fp32:.6f}")



MAE (INT8): 0.270235
MSE (INT8): 0.132914
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
MAE (FP32): 0.102499
MSE (FP32): 0.028381
